# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [1]:
import matplotlib.pyplot as plt
import csv
import os
import pickle
import skimage
import numpy as np
from sklearn.utils import shuffle
import cv2


#########################
# Initialize constants
#########################
training_file = 'data/train.p'
validation_file='data/train.p'
testing_file = 'data/test.p'
imgs_path = "new_images/"
labels_path = "signnames.csv"


#########################
# Load data
#########################
with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
        
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']




######################################################################################
#   flip image from left to right and add them to new data
######################################################################################
def flip(images):
    flipped = np.zeros((images.shape[0], 32, 32, 3))
    for i in range(0, images.shape[0]):        
            flipped[i] = cv2.flip(images[i], 1)        
    return images



X_train_flipped = flip(X_train)
X_valid_flipped = flip(X_valid)
X_test_flipped = flip(X_test)



######################################################################################
#   Add
######################################################################################
X_train = np.concatenate((X_train_flipped, X_train), axis=0)
X_test = np.concatenate((X_test_flipped, X_test), axis=0)
X_valid = np.concatenate((X_valid_flipped, X_valid), axis=0)

y_train = np.concatenate((y_train, y_train), axis=0)
y_test = np.concatenate((y_test, y_test), axis=0)
y_valid = np.concatenate((y_valid, y_valid), axis=0)

print(X_train.shape[0])
print(X_test.shape[0])
print(X_valid.shape[0])


######################################################################################
#   splitting and shuffling dataset directly in the pandas
######################################################################################
def split(X, y):
    msk = np.random.rand(len(X)) < 0.8

    X_train = X[msk]
    X_valid = X[~msk]

    y_train = y[msk]
    y_valid = y[~msk]

    return X_train, X_valid, y_train, y_valid

X_train, X_valid, y_train, y_valid = split(X_train, y_train)











FileNotFoundError: [Errno 2] No such file or directory: 'data/train.p'

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
#Number of training examples
n_train = X_train.shape[0]

#Number of validation examples
n_validation = X_valid.shape[0]

#Number of testing examples.
n_test = X_test.shape[0]

#What's the shape of an traffic sign image?
image_shape = X_train.shape[1], X_train.shape[2]

#Number of unique classes/labels there are in the dataset.
n_classes = y_valid[y_valid.shape[0]-1]+1

#########################
# Print basic stats
#########################
print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
import matplotlib.pyplot as plt
import random
%matplotlib inline

#########################
# Print histograms
#########################

# train / test / valid
h1 = plt.hist(y_train, bins='auto', alpha=0.5, color=['green'], label='train')
h2 = plt.hist(y_test, bins='auto', alpha=0.5, color=['blue'], label='test')
h3 = plt.hist(y_valid, bins='auto', alpha=0.5, color=['red'], label='validation')



plt.title("Datasets distribution")
plt.xlabel('Category ID')
plt.ylabel('Number of occurences')

plt.legend(shadow=True, fancybox=True)
plt.grid()

plt.show()


#########################
# Show example image
#########################

print("-------------------------------------------------------------------------")
fig, ax = plt.subplots()
im = plt.imshow(X_train[0][:][:][:], interpolation='nearest')
plt.title('Example image')

ax.axis('off')
plt.show()


#########################
# Show example image
#########################

print("-------------------------------------------------------------------------")
fig, ax = plt.subplots()
im = plt.imshow(X_train[999][:][:][:], interpolation='nearest')
plt.title('Example image')

ax.axis('off')
plt.show()





----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
#########################
# preprocess array of images
# We are going to convert images to grayscale and normalize (-1, +1)
#########################
def preprocess_images(X):
    import scipy.misc
    import scipy.ndimage
    from skimage import data, exposure, img_as_float
    
    R = X[:,:, :, 0]
    G = X[:,:, :, 1]
    B = X[:, :, :, 2]
    
    # convert to grayscale
    X_preprocessed = R * 299. / 1000 + G * 587. / 1000 + B * 114. / 1000
    
    # normalize
    X_preprocessed = ((X_preprocessed - 128)/128).astype(np.float32)
    
    eq_image = exposure.equalize_adapthist(X_preprocessed[0][:][:][:], 1, clip_limit=0.01)
    plt.imshow(eq_image, cmap='gray')
    
    # change dimensions to fit
    X_preprocessed = np.expand_dims(X_preprocessed, axis=4)
    
    return X_preprocessed



#########################
# Preprocess all datasets
#########################
X_train_preprocessed = preprocess_images(X_train)
X_valid_preprocessed = preprocess_images(X_valid)
X_test_preprocessed = preprocess_images(X_test)

print("processed")





### Model Architecture

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import flatten

BATCH_SIZE = 256


#########################
# Model definition
#########################
def LeNet(x):    
    
   

        mu = 0
        sigma = 0.1
        # We will create 2 convolution layers
        # Layer Convolutional - Input = 32x32x1. Output = 28x28x6.
        with tf.name_scope("Convolution_Layer_1"):
            conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 6), mean = mu, stddev = sigma))
            conv1_b = tf.Variable(tf.zeros(6), name="conv1_b")
            conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b
            # Dropout layer - prevent ovefit
            conv1 = tf.nn.dropout(conv1, keep_prob)
            # RELU activation
            conv1 = tf.nn.elu(conv1)
            # Pooling. Input = 28x28x6. Output = 14x14x6.
            conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
            
        # Layer : Convolutional. Output = 10x10x16.
        with tf.name_scope("Convolution_Layer_2"):

            conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
            conv2_b = tf.Variable(tf.zeros(16))
            conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b                     
            # Dropout layer - prevent ovefit
            conv2 = tf.nn.dropout(conv2, keep_prob)
            # RELU activation
            conv2 = tf.nn.elu(conv2)
            # Pooling. Input = 10x10x16. Output = 5x5x16.
            conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
            
            # Squash image. Input = 5x5x16. Output = 400
            fc0   = flatten(conv2)
            
            
        # We will create 3 fully connected layers    
        # Input = 400. Output = 120.     
        with tf.name_scope("Fully_ConnectedLayer_1"):
            fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
            fc1_b = tf.Variable(tf.zeros(120))
            fc1   = tf.matmul(fc0, fc1_W) + fc1_b
            fc1   = tf.nn.elu(fc1)
        # Input = 120. Output = 84. 
        with tf.name_scope("Fully_Connected_Layer_2"):

            fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
            fc2_b  = tf.Variable(tf.zeros(84))
            fc2    = tf.matmul(fc1, fc2_W) + fc2_b
            fc2    = tf.nn.elu(fc2)
                
        # Input = 84. Output = n_classes (43).        
        with tf.name_scope("Fully_Connected_Layer_3"):

            fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, n_classes), mean = mu, stddev = sigma))
            fc3_b  = tf.Variable(tf.zeros(n_classes))
            logits = tf.matmul(fc2, fc3_W) + fc3_b

        return logits



############################################
# Create graph
############################################    


############################################
# Define placeholders 
############################################
x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, n_classes)
keep_prob = tf.placeholder(tf.float32)



############################################
# Define network and training operations
############################################
logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer()
    
training_operation = optimizer.minimize(loss_operation)
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))

with tf.name_scope('summaries'):
    accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('acc', accuracy_operation)
    
saver = tf.train.Saver()
summary_op = tf.summary.merge_all()





### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
############################################
# Function evaluating accuracy of the prediction over dataset
############################################
def evaluate(X_data, y_data):
    num_examples = len(X_data)

    total_accuracy = 0
    sess = tf.get_default_session()
    # make batches of images and compute accuracy
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]

        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1})
    
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples 







############################################
# Main train model loop
############################################.
EPOCHS = 10000
model_name = './model/lenet'
model_path = './model/'
model_meta_path = './model/lenet.meta'
max_validation_accuracy = 0.0
number_of_epochs_without_improvement = 0

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    writer = tf.summary.FileWriter("output", sess.graph)
    
    
    num_examples = len(X_train_preprocessed)
    saver = tf.train.import_meta_graph(model_meta_path)
    saver.restore(sess, tf.train.latest_checkpoint(model_path))
    print("Training...")
    for i in range(EPOCHS):
        for offset in range(0, n_train, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train_preprocessed[offset:end], y_train[offset:end]
            summary = sess.run([training_operation, accuracy_operation], feed_dict={x: batch_x, y: batch_y, keep_prob: 0.5})     
            #writer.add_summary(summary, i)

         
        
        validation_accuracy = evaluate(X_valid_preprocessed, y_valid)
        test_accuracy = evaluate(X_test_preprocessed, y_test)
        train_accuracy = evaluate(X_train_preprocessed, y_train)

        
        # print basic summary
        print("EPOCH {} ...".format(i+1))
        print("Validation Set Accuracy = {:.5f}".format(validation_accuracy))
        print("Test Set Accuracy = {:.5f}".format(test_accuracy))
        print("Train Set Accuracy = {:.5f}".format(train_accuracy))
        
        # If we have best model according the validation accuracy, we will save it to model "lenet"
        if (max_validation_accuracy < validation_accuracy):
            max_validation_accuracy = validation_accuracy
            saver.save(sess, model_name)
            print("Model saved")
        else:
            number_of_epochs_without_improvement = number_of_epochs_without_improvement + 1
        if (number_of_epochs_without_improvement >= 1):
            print("ending")
            break
            
    writer.close()






---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
from os import listdir
from PIL import Image as PImage

############################################
# Load images from the directory - again, just for case we do not want to training loop
############################################
def loadImages(path):
    # return array of images
    import os, fnmatch
    imagesList = fnmatch.filter(os.listdir(path), '*.jpg')
    loadedImages = []
    for image in imagesList:
        
        img = PImage.open(path + image)
        loadedImages.append(img)

    return loadedImages

############################################
# Provide routine for loading labels from csv file
############################################
def loadLabelNames(path):
    import csv;
    signames = []
    with open(path) as csvfile:
            csvreader = csv.reader(csvfile, delimiter=',', quotechar='"')
            
            next(csvreader, None)
            for row in csvreader:   
                signames.append(row)
    return signames
                

############################################
# Show new example images
############################################
imgs = loadImages(imgs_path)
signames = loadLabelNames(labels_path)
new_imgs = np.zeros((5, 32, 32, 3))
new_names = ["Road work", "Stop", "General caution", "Speed limit (60km/h)","Pedestrians"]
index = 0
        
for img in imgs:
        img = img.resize((32,32), PImage.ANTIALIAS)
        
        im_arr = np.fromstring(img.tobytes(), dtype=np.uint8)
        
        im_arr = im_arr.reshape((1, img.size[1], img.size[0], 3))    
       
        plt.imshow(img)
        new_imgs[index] = im_arr
        
        plt.show()
        index = index + 1
       



### Predict the Sign Type for Each Image

In [ ]:
############################################
# We will restore trained model, load all new images and compare prediction with the
# ground truth sign. If sign is correct, we will increase accuracy.
############################################

probabs_i = []
indicies_i = []
index = 0
pred_acc = 0.0

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    new_saver = tf.train.import_meta_graph('./model/lenet.meta')
   
    new_saver.restore(sess, tf.train.latest_checkpoint('./model/'))

    # your images in an array
    imgs = loadImages(imgs_path)
    signames = loadLabelNames(labels_path)
    
    plt.figure("images")
    
    for img in imgs:
        img = img.resize((32,32), PImage.ANTIALIAS)
        
        im_arr = np.fromstring(img.tobytes(), dtype=np.uint8)
        
        im_arr = im_arr.reshape((1, img.size[1], img.size[0], 3))    
        
        # you can show every image
        plt.subplot(212)
        plt.imshow(img)
        plt.show()
       

        im_arr = preprocess_images(im_arr)
       
        predicted = sess.run(logits,feed_dict={x: im_arr, keep_prob: 1})

        soft = sess.run(tf.nn.softmax(predicted))

        
        probabs, indicies = sess.run(tf.nn.top_k(soft, k=5))
        
        probabs_i.append(probabs)
        indicies_i.append(indicies)


       
        for j in range(0, predicted.shape[0]):
            p = signames[np.argmax(soft[j], axis = 0)]
            print("Predicted: "+p[1]) 
            print("Ground truth: "+new_names[index]) 
            if new_names[index].lower() == p[1].lower():
                print("Correct prediction")
                pred_acc += 1.0
            else:
                print("Uncorrect prediction")

            print("------------------------------------------")
        index = index + 1
    pred_acc = pred_acc / index
    print("Final accuracy is: " + str(pred_acc * 100) + "%")
          

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.
print("Top 5 softmax probabs for each image:")
for i in range(probabs.shape[1]):
    print("---------------------------------------- Image number: "+str(i+1)+"-------------------------------")
    softmax_arr = probabs_i[i][0][:]
    for j in range(softmax_arr.shape[0]):
        print("{0:.3f}".format((softmax_arr[j]*100)) + " %") 
        

    

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")